In [1]:
from PIL import Image
from skimage import io
import matplotlib.pyplot as plt
%matplotlib notebook
plt.ioff()
import tifffile, csv, numpy as np
from skimage.morphology import closing, opening, square, disk, ball
from skimage.segmentation import watershed
from skimage.filters import threshold_otsu, sobel, threshold_local, threshold_niblack
from skimage.measure import label, regionprops
from skimage.color import label2rgb
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
import mahotas as mh
import cv2, skimage
import os, copy
from skimage.draw import disk as pdisk

In [2]:
directory = "/media/ula/D/ppp/fos_gfp_tmaze2/processing/"
path_comm = "m{}{}_tst_sca.tif"
out_name = "result_m{}{}.tif"
result_file = "result_label_m{}{}.csv"
bgr_file = "result_bg_m{}{}.csv"


roi_diameter = [8,7,4]

field_names_coords = ['X', 'Y', 'Z']
scale_coeff = {
    'xy': 1.2, 
    'z': 2
}
radius_microns = 5
min_dist = 1.2*radius_microns


tolerance = radius_microns

In [3]:
def dist(centroid1, centroid2):
    return np.sqrt((scale_coeff['xy']*(centroid1[0]-centroid2[0]))**2
                   +(scale_coeff['xy']*(centroid1[1]-centroid2[1]))**2
                   +(scale_coeff['z']*(centroid1[2]-centroid2[2]))**2)


In [4]:
def calculate_markers(stack_slice, otsu_offset):
    elev_map_ = np.array(sobel(stack_slice))
    
    thre_otsu = threshold_otsu(stack_slice)
    thre_val = threshold_local(stack_slice, 75, offset=-otsu_offset*thre_otsu)
    stack_slice[stack_slice > thre_val] = 255
    stack_slice[stack_slice <= thre_val] = 0
    
    thresh = stack_slice
    kernel = disk(1)
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 1)
    sure_bg = cv2.dilate(opening,kernel,iterations=1)
    dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,3)
    ret, sure_fg = cv2.threshold(dist_transform,0.2*dist_transform.max(),255,0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)
    ret, markers_ = cv2.connectedComponents(sure_fg)
    markers_ = markers_+1
    markers_[unknown==255] = 0
    return markers_, opening, elev_map_

In [5]:
def prepare_img_for_watershed(image_ts, out_name, otsu_offset):
    elev_map = np.empty_like(image_ts)
    markers = np.empty_like(image_ts)

    for idx, im in enumerate(image_ts):
        markers_, opening, elev_map_ = calculate_markers(im, otsu_offset)
        markers[idx] = markers_
        image_ts[idx] = opening
        elev_map[idx] = elev_map_

    with tifffile.TiffWriter(out_name) as tiff:   
        for im in image_ts:
            tiff.save(im)
    return elev_map, markers

In [6]:
def watershed_label_img(elev_map, markers, image_ts):
    

    labels = label(watershed(-elev_map, markers, connectivity=2))
    #labels = label(image_ts)

    return labels



In [7]:
def identify_cells(label_image):
    cells = []
    total = len(regionprops(label_image))
    with open(result_file,"w") as result:
        wtr = csv.writer( result )
        wtr.writerow( field_names_coords )
        for region in regionprops(label_image):
            if region.area > 20:
                coords = np.flip(region.centroid).astype(int)+np.array([1,1,1])
                cells.append(coords)
                wtr.writerow(coords)
                
    unique_cells = []
    print("non unique cells ", len(cells))
    for idx_t, c1 in enumerate(cells):
        if idx_t% 10000 == 0:
            print(idx_t/total*100, '%')
        idx = 0
        for c2 in unique_cells:
            if dist(c1, c2) < min_dist:
                break;
            idx += 1
        if idx == len(unique_cells):
            unique_cells.append(c1)
        else:
            unique_cells[idx] = ((c1+c2)/2).astype(int)
              
    return unique_cells

In [8]:
def save_result(result_file, cells, header=None):
    print(result_file)
    with open(result_file,"w") as result:
        wtr = csv.writer( result )
        if header != None:
            wtr.writerow( header )
        for cell in cells:
            wtr.writerow(cell)

In [9]:
def find_cells(mouse, scan_code, otsu_offset):
    source_path = directory + path_comm.format(mouse, scan_code)
    result_img_path = directory + out_name.format(mouse, scan_code)
    result_csv_path = directory + result_file.format(mouse, scan_code)
    
    image_ts = io.imread(source_path)
    
    elev_map, markers = prepare_img_for_watershed(image_ts, result_img_path, otsu_offset)
    label_image = watershed_label_img(elev_map, markers, image_ts)
    cells = identify_cells(label_image)
    print(mouse, scan_code, len(cells))
    save_result(result_csv_path, cells, field_names_coords)
    return cells, image_ts
    

In [10]:
def estimate_background(cells, img):
    background_size = 1000
    chosen_count = 0
    
    for cell in cells:
        z = cell[2]
        for i in range(-2,3):
            if(z+i >= 0 and z+i < img.shape[0]):
                diameter = roi_diameter[abs(i)] + roi_diameter[0]
                rad = diameter/2
                disk = pdisk(cell[:2], rad, shape=img[0].shape)
                img[z+i][disk] = -1
    
    random_centroids = []
    while chosen_count < background_size:
        indices = np.array([], dtype=int)
        for i in img.shape:
            indices =  np.append(indices, np.random.randint(0, high=i))
        if img[indices[0]][indices[1]][indices[2]] > 0:
            random_centroids.append(indices)
            chosen_count +=1
    return random_centroids

In [11]:
cells, img = find_cells(3, 's1_r1', 0.3)

non unique cells  4348
0.0 %
3 s1_r1 1666
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_label_m3s1_r1.csv


In [12]:
random_centroids = estimate_background(cells, copy.copy(img))

In [13]:
save_result(directory + bgr_file.format(3, 's1_r1'), random_centroids, field_names_coords)

/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_bg_m3s1_r1.csv


In [14]:
img.shape

(80, 512, 512)

In [15]:
reg_code = '_r1'
for i in range(3):
    cells, img = find_cells(3, 's' + str(1 + i) + reg_code, 0.3)
    random_centroids = estimate_background(cells, copy.copy(img))
    save_result(directory + bgr_file.format(3, 's' + str(1 + i) + reg_code), random_centroids, field_names_coords)

non unique cells  4348
0.0 %
3 s1_r1 1666
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_label_m3s1_r1.csv
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_bg_m3s1_r1.csv
non unique cells  7262
0.0 %
3 s2_r1 2550
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_label_m3s2_r1.csv
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_bg_m3s2_r1.csv
non unique cells  6292
0.0 %
3 s3_r1 2231
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_label_m3s3_r1.csv
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_bg_m3s3_r1.csv


In [16]:
mice = ['2', '3', '4', '7', '8', '12']
reg_code = '_r1'

starting_sess = 1

for mouse in mice:
    for i in range(3):
        cells, img = find_cells(mouse, 's' + str(starting_sess + i) + reg_code, 0.3)
        random_centroids = estimate_background(cells, copy.copy(img))
        save_result(directory + bgr_file.format(mouse, 's' + str(starting_sess + i) + reg_code), random_centroids, field_names_coords)

mouse = '9'
reg_code = '_r2'

for i in range(3):
    cells, img = find_cells(mouse, 's' + str(starting_sess + i) + reg_code, 0.3)
    random_centroids = estimate_background(cells, copy.copy(img))
    save_result(directory + bgr_file.format(mouse, 's' + str(starting_sess + i) + reg_code), random_centroids, field_names_coords)

non unique cells  7976
0.0 %
2 s1_r1 2873
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_label_m2s1_r1.csv
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_bg_m2s1_r1.csv
non unique cells  7229
0.0 %
2 s2_r1 2717
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_label_m2s2_r1.csv
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_bg_m2s2_r1.csv
non unique cells  6808
0.0 %
2 s3_r1 2566
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_label_m2s3_r1.csv
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_bg_m2s3_r1.csv
non unique cells  4348
0.0 %
3 s1_r1 1666
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_label_m3s1_r1.csv
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_bg_m3s1_r1.csv
non unique cells  7262
0.0 %
3 s2_r1 2550
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_label_m3s2_r1.csv
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_bg_m3s2_r1.csv
non unique cells  6292
0.0 %
3 s3_r1 2231
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_label_m3s3_r1.csv
/media/ula/D/pp

In [17]:
def read_coords(filename):
    ret = []
    with open(filename,"r") as source:
        rdr = csv.reader(source)
        next(rdr)
        for row in rdr:
            ret.append(row)
    return np.array(ret).astype(int)


In [18]:
path_comm = "m{}{}_tst_sa.tif"
out_name = "result_sa_m{}{}.tif"

In [19]:
reg_code = '_r1'

for mouse in mice:
    for i in range(3):
        find_cells(mouse, 's' + str(starting_sess + i) + reg_code, 0.3)

mouse = '9'
reg_code = '_r2'

for i in range(3):
    find_cells(mouse, 's' + str(starting_sess + i) + reg_code, 0.3)

non unique cells  11025
0.0 %
44.90950734270445 %
2 s1_r1 3737
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_label_m2s1_r1.csv
non unique cells  9975
0.0 %
2 s2_r1 3443
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_label_m2s2_r1.csv
non unique cells  9090
0.0 %
2 s3_r1 3158
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_label_m2s3_r1.csv
non unique cells  9247
0.0 %
3 s1_r1 3303
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_label_m3s1_r1.csv
non unique cells  10216
0.0 %
49.63271788763153 %
3 s2_r1 3472
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_label_m3s2_r1.csv
non unique cells  9738
0.0 %
3 s3_r1 3311
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_label_m3s3_r1.csv
non unique cells  5149
0.0 %
4 s1_r1 1836
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_label_m4s1_r1.csv
non unique cells  6065
0.0 %
4 s2_r1 2161
/media/ula/D/ppp/fos_gfp_tmaze2/processing/result_label_m4s2_r1.csv
non unique cells  6423
0.0 %
4 s3_r1 2282
/media/ula/D/ppp/fos_gfp_tmaze2/proc